In [ ]:
!pip install opendatasets

## Importing libs and downloading Dataset

In [17]:
import opendatasets as od
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,f1_score,ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np

def train_test_split(X,y,test_size= 0.2,random_state=None,shuffle: bool = True):

    n = len(X)
    indices = np.arange(n)

    if shuffle:
        rng = np.random.default_rng(random_state)
        rng.shuffle(indices)

    test_count = int(n * test_size)
    test_idx = indices[:test_count]
    train_idx = indices[test_count:]

    X_train = X.iloc[train_idx].reset_index(drop=True)
    X_test  = X.iloc[test_idx].reset_index(drop=True)
    y_train = y.iloc[train_idx].reset_index(drop=True)
    y_test  = y.iloc[test_idx].reset_index(drop=True)

    return X_train, X_test, y_train, y_test


In [ ]:
od.download("https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset")

# EXPLORING DATASET

In [5]:
df = pd.read_csv("/content/heart-disease-dataset/heart.csv")

In [6]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [9]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [13]:
sc = StandardScaler()
sc.fit(df.drop("target",axis=1))
X_scale = sc.transform(df.drop("target",axis=1))
y = df["target"]

In [46]:
df['target'].value_counts()

,count
target,
1,526
0,499


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.33, random_state=42)

In [ ]:
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
X_std[X_std == 0] = 1

# SUPERVISED TASK

## Function Defintions

In [ ]:
def sigmoid(z):
  return 1/(1+np.exp(-z))   

In [ ]:
def gradient_descent(X,y,params,lr = 0.05,iter=1000):
  X = (X-X_mean)/X_std
  X = np.c_[np.ones((X.shape[0], 1)), X]
  m = X.shape[0]
  cost_history = []
  for i in range(iter):
    pred = sigmoid(X@params)
    error = pred-y
    grad = (X.T@(error))/m
    cost = (-1/m)*np.sum(y*np.log(pred+1e-9)+(1-y)*np.log(1-pred+1e-9))
    cost_history.append(cost)
    params = params - lr*(grad)
  return params,cost_history


In [ ]:
def predict(W,X):
  X = (X-X_mean)/X_std
  X = np.c_[np.ones((X.shape[0], 1)), X]
  pred = X@W
  return (pred>=0.5).astype(int)

## Training Custom Logisitic Regression

In [ ]:
W,costs = gradient_descent(X_train,y_train,np.zeros((X_train.shape[1]+1,1)),lr=0.05,iter=1000)

## Training Sklearn's logistic

In [ ]:
y_train = y_train.reshape(-1)

In [ ]:
model  = LogisticRegression(max_iter=5000).fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

## Evaluation of Models

#### Custom

In [ ]:
print(precision_score(y_test,predict(W,X_test)))
print(accuracy_score(y_test,predict(W,X_test)))
print(recall_score(y_test,predict(W,X_test)))
print(f1_score(y_test,predict(W,X_test)))
print(confusion_matrix(y_test,predict(W,X_test)))

#### Sklearn

In [ ]:
print(precision_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(recall_score(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

# UNSUPERVISED TASK

In [ ]:
import numpy as np

def kmeans(X, k, max_iter=100, tol=1e-4, random_state=None):
    rng = np.random.default_rng(random_state)
    X = np.asarray(X)
    centroids = X[rng.choice(len(X), k, replace=False)]
    labels = None
    for _ in range(max_iter):
        distances = np.linalg.norm(X[:, None] - centroids[None, :], axis=2)
        labels = distances.argmin(axis=1)
        new_centroids = np.array([
            X[labels == i].mean(axis=0) for i in range(k)
        ])
        if np.linalg.norm(new_centroids - centroids) < tol:
            break
        centroids = new_centroids

    return labels, centroids


In [ ]:
kmeans(X_train,5)